## Reading and analyze the data. 


In [20]:
import pandas as pd
import numpy as np
import pandas as pd
from itertools import chain, combinations, permutations

path = input("Give the name of dataset file: ")

#if path == None: 
#  path = '/content/drive/MyDrive/GroceryStore.csv'

#path = '/content/drive/MyDrive/GroceryStore.csv'

df = pd.read_csv(path,names = ['items'], sep = ',')
df.head(5)

df_1  = df['items'].apply(lambda x : x.split(','))


liste = []

for i in range(len(df_1)):
    x = df_1[i]
    print(x)
    liste.append(x)

#converting dataset to binary format 
from mlxtend.preprocessing import TransactionEncoder
encoder = TransactionEncoder()
pred = encoder.fit_transform(liste)
df = pd.DataFrame(pred.astype("int"),columns = encoder.columns_)
df.head(21)

Give the name of dataset file: GroceryStore.csv
['MILK', 'BREAD', 'BISCUIT']
['BREAD', 'MILK', 'BISCUIT', 'CORNFLAKES']
['BREAD', 'TEA', 'BOURNVITA']
['JAM', 'MAGGI', 'BREAD', 'MILK']
['MAGGI', 'TEA', 'BISCUIT']
['BREAD', 'TEA', 'BOURNVITA']
['MAGGI', 'TEA', 'CORNFLAKES']
['MAGGI', 'BREAD', 'TEA', 'BISCUIT']
['JAM', 'MAGGI', 'BREAD', 'TEA']
['BREAD', 'MILK']
['COFFEE', 'COCK', 'BISCUIT', 'CORNFLAKES']
['COFFEE', 'COCK', 'BISCUIT', 'CORNFLAKES']
['COFFEE', 'SUGER', 'BOURNVITA']
['BREAD', 'COFFEE', 'COCK']
['BREAD', 'SUGER', 'BISCUIT']
['COFFEE', 'SUGER', 'CORNFLAKES']
['BREAD', 'SUGER', 'BOURNVITA']
['BREAD', 'COFFEE', 'SUGER']
['BREAD', 'COFFEE', 'SUGER']
['TEA', 'MILK', 'COFFEE', 'CORNFLAKES']


,BISCUIT,BOURNVITA,BREAD,COCK,COFFEE,CORNFLAKES,JAM,MAGGI,MILK,SUGER,TEA
0,1,0,1,0,0,0,0,0,1,0,0
1,1,0,1,0,0,1,0,0,1,0,0
2,0,1,1,0,0,0,0,0,0,0,1
3,0,0,1,0,0,0,1,1,1,0,0
4,1,0,0,0,0,0,0,1,0,0,1
5,0,1,1,0,0,0,0,0,0,0,1
6,0,0,0,0,0,1,0,1,0,0,1
7,1,0,1,0,0,0,0,1,0,0,1
8,0,0,1,0,0,0,1,1,0,0,1
9,0,0,1,0,0,0,0,0,1,0,0


## Find all possible 2, 3, 4 and 5-itemsets
I have chosen support value to be 10% meaning 0.1 for 1 and 2 iteration and 0.01 for 3rd and 4th iteration. 

In [21]:
from itertools import chain

def ar_iterations(data, num_iter = 1, support_value = 0.1, iterationIndex = None):
    
    # Next Iterations
    def ar_calculation(iterationIndex = iterationIndex): 
        # Calculation of support value
        value = []
        for i in range(0, len(iterationIndex)):
            result = data.T.loc[iterationIndex[i]].sum() 
            result = len(result[result == data.T.loc[iterationIndex[i]].shape[0]]) / data.shape[0]
            value.append(result)
        # Bind results
        result = pd.DataFrame(value, columns = ["Support"])
        result["index"] = [tuple(i) for i in iterationIndex]
        result['length'] = result['index'].apply(lambda x:len(x))
        result = result.set_index("index").sort_values("Support", ascending = False)
        # Elimination by Support Value
        result = result[result.Support > support_value]
        return result    
    
    # First Iteration
    first = pd.DataFrame(df.T.sum(axis = 1) / df.shape[0], columns = ["Support"]).sort_values("Support", ascending = False)
    first = first[first.Support > support_value]
    first["length"] = 1
    
    if num_iter == 1:
        res = first.copy()
        
    # Second Iteration
    elif num_iter == 2:
        
        second = list(combinations(first.index, 2))
        second = [list(i) for i in second]
        res = ar_calculation(second)
        
    # All Iterations > 2
    else:
        nth = list(combinations(set(list(chain(*iterationIndex))), num_iter))
        nth = [list(i) for i in nth]
        res = ar_calculation(nth)
    
    return res



iteration1 = ar_iterations(df, num_iter=1, support_value=0.1)
iteration1



,Support,length
BREAD,0.65,1
COFFEE,0.40,1
BISCUIT,0.35,1
TEA,0.35,1
CORNFLAKES,0.30,1
SUGER,0.30,1
MAGGI,0.25,1
MILK,0.25,1
BOURNVITA,0.20,1
COCK,0.15,1


In [22]:
iteration2 = ar_iterations(df, num_iter=2, support_value=0.1)
iteration2

,Support,length
index,,
"(COFFEE, CORNFLAKES)",0.20,2
"(BREAD, TEA)",0.20,2
"(TEA, MAGGI)",0.20,2
"(BREAD, SUGER)",0.20,2
"(BREAD, BISCUIT)",0.20,2
"(BREAD, MILK)",0.20,2
"(COFFEE, SUGER)",0.20,2
"(BREAD, COFFEE)",0.15,2
"(COFFEE, COCK)",0.15,2


In [23]:
iteration3 = ar_iterations(df, num_iter=3, support_value=0.01,
              iterationIndex=iteration2.index)
iteration3

,Support,length
index,,
"(BREAD, BOURNVITA, TEA)",0.10,3
"(BREAD, TEA, MAGGI)",0.10,3
"(COCK, BISCUIT, COFFEE)",0.10,3
"(BISCUIT, TEA, MAGGI)",0.10,3
"(COCK, COFFEE, CORNFLAKES)",0.10,3
"(SUGER, COFFEE, BREAD)",0.10,3
"(COCK, BISCUIT, CORNFLAKES)",0.10,3
"(BISCUIT, COFFEE, CORNFLAKES)",0.10,3
"(BISCUIT, BREAD, MILK)",0.10,3


In [24]:

iteration4 = ar_iterations(df, num_iter=4, support_value=0.01,
              iterationIndex=iteration3.index)
iteration4

,Support,length
index,,
"(COCK, BISCUIT, COFFEE, CORNFLAKES)",0.10,4
"(BISCUIT, BREAD, TEA, MAGGI)",0.05,4
"(BISCUIT, BREAD, MILK, CORNFLAKES)",0.05,4
"(COFFEE, MILK, CORNFLAKES, TEA)",0.05,4


## Coding Apriori Algorithm from scratch

In [25]:


def APRIORI(data, min_support, max_length = 4):
  #min_support is the minimun threshold support set by us 
  #max_length is the maximun number of items to be included in itemset

  #step 1: create a dic to store support of an itemset
  support = {}
  L = set(data.columns)

  #step 2: generate combinations of items with len i in the ith iteration
  for i in range(1, max_length+1):
    c = list(combinations(L,i))

    #reset L for the next (i+1)th iteration
    L = set()
  #step 3: iterate through each item in "c"
    for j in list(c):
      sup = data.loc[:,j]. product(axis=1).sum()/len(data.index)
      if sup >= min_support:
        support[j] = sup

        #appending frequent itemsets in list L, already reset list L 
        L = set(set(L)| set(j))

  #step 4: create data frame with cols "items" , "support"
  result = pd.DataFrame(list(support.items()), columns = ["Items", "Support"])
  return(result)





In [34]:
#finding frequent itemset with min support of 4%, 10%, 15%, 20% and 25% for testing 
#after many trial and error min support of 20% meaning 0.2 gives the most best association rules


my_frequent_itemset = APRIORI(df, 0.2, 4)
print(my_frequent_itemset)
my_frequent_itemset.sort_values(by = "Support", ascending = False)


                   Items  Support
0               (SUGER,)     0.30
1             (BISCUIT,)     0.35
2              (COFFEE,)     0.40
3               (BREAD,)     0.65
4           (BOURNVITA,)     0.20
5                (MILK,)     0.25
6          (CORNFLAKES,)     0.30
7                 (TEA,)     0.35
8               (MAGGI,)     0.25
9        (SUGER, COFFEE)     0.20
10        (SUGER, BREAD)     0.20
11      (BISCUIT, BREAD)     0.20
12  (COFFEE, CORNFLAKES)     0.20
13         (BREAD, MILK)     0.20
14          (BREAD, TEA)     0.20
15          (TEA, MAGGI)     0.20


,Items,Support
3,"(BREAD,)",0.65
2,"(COFFEE,)",0.40
1,"(BISCUIT,)",0.35
7,"(TEA,)",0.35
0,"(SUGER,)",0.30
6,"(CORNFLAKES,)",0.30
5,"(MILK,)",0.25
8,"(MAGGI,)",0.25
4,"(BOURNVITA,)",0.20
9,"(SUGER, COFFEE)",0.20


In [27]:
import pandas as pd

#Apply association rules to the frequent itemset obtained in the previous algorithm. Set minimum threshold confidence to 50%.

def ASSOCIATION(df, min_threshold):
  #step 1: creating required variable 
  support = pd.Series(df.Support.values, index = df.Items).to_dict()
  data = []
  L = df.Items.values
  #step 2: generating rule using permutation 
  p = list(permutations(L,2))

  #iterating though all possible rule 

  for i in p: 

    #if Antecedent of rule is subset of consequents, LHS -> RHS then valid rule
    if set(i[0]).issubset(i[1]):
      conf = support[i[1]] / support[i[0]]
      #print(i, conf)
      if conf > min_threshold:
        j = i[1][not i[1].index(i[0][0])]
        lift = support[i[1]] / (support[i[0]]*support[(j,)])
        leverage = support[i[1]] - (support[i[0]] * support[(j,)])
        #convection = "ignore" # trouble shoot. denominator very close to 0. 
        data.append([i[0], (j,), support[i[0]], support[(j,)], support[i[1]], conf, lift, leverage])



  #step 3: 
  result = pd.DataFrame(data, columns= ["antecedents", "consequents", "antecedent support", "consequent support","support","confidence", "lift","leverage"])
  return(result)

                            


In [28]:
#rule with minimum confidence = 50% 
rules_found = ASSOCIATION(my_frequent_itemset, 0.5)
rules_found

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage
0,"(SUGER,)","(COFFEE,)",0.30,0.40,0.2,0.666667,1.666667,0.0800
1,"(SUGER,)","(BREAD,)",0.30,0.65,0.2,0.666667,1.025641,0.0050
2,"(BISCUIT,)","(BREAD,)",0.35,0.65,0.2,0.571429,0.879121,-0.0275
3,"(MILK,)","(BREAD,)",0.25,0.65,0.2,0.800000,1.230769,0.0375
4,"(CORNFLAKES,)","(COFFEE,)",0.30,0.40,0.2,0.666667,1.666667,0.0800
5,"(TEA,)","(BREAD,)",0.35,0.65,0.2,0.571429,0.879121,-0.0275
6,"(TEA,)","(MAGGI,)",0.35,0.25,0.2,0.571429,2.285714,0.1125
7,"(MAGGI,)","(TEA,)",0.25,0.35,0.2,0.800000,2.285714,0.1125


In [29]:
#writing the results in a text file 

rules_found.to_csv(r'Items.txt', header=None, index=None, sep=' ', mode='a')

In [30]:
# Read file
#We can read the file to see if it worked:

with open('Items.txt', 'r') as testwritefile:
    print(testwritefile.read())

This file will contain outputs:('SUGER',) ('COFFEE',) 0.3 0.4 0.2 0.6666666666666667 1.6666666666666667 0.08000000000000002
('SUGER',) ('BREAD',) 0.3 0.65 0.2 0.6666666666666667 1.0256410256410258 0.0050000000000000044
('BISCUIT',) ('BREAD',) 0.35 0.65 0.2 0.5714285714285715 0.8791208791208792 -0.02749999999999997
('MILK',) ('BREAD',) 0.25 0.65 0.2 0.8 1.2307692307692308 0.037500000000000006
('CORNFLAKES',) ('COFFEE',) 0.3 0.4 0.2 0.6666666666666667 1.6666666666666667 0.08000000000000002
('TEA',) ('BREAD',) 0.35 0.65 0.2 0.5714285714285715 0.8791208791208792 -0.02749999999999997
('TEA',) ('MAGGI',) 0.35 0.25 0.2 0.5714285714285715 2.285714285714286 0.11250000000000002
('MAGGI',) ('TEA',) 0.25 0.35 0.2 0.8 2.285714285714286 0.11250000000000002
('SUGER',) ('COFFEE',) 0.3 0.4 0.2 0.6666666666666667 1.6666666666666667 0.08000000000000002
('SUGER',) ('BREAD',) 0.3 0.65 0.2 0.6666666666666667 1.0256410256410258 0.0050000000000000044
('BISCUIT',) ('BREAD',) 0.35 0.65 0.2 0.5714285714285715 0.

## Interpreting Results: 



We get MAGGI and TEA Association Rule with Confidence Score of 80% with 2.28 Lift Score. Meaning, consumers who purchase MAGGI are 2.28 times more likely to purchase TEA than randomly chosen customers.

Similarly, We get MILK and BREAD Association Rule. Confidence Score 80% with Lift Score of 1.23. Because the lift score is too close to 1, they have very slight possibility of being purchased together. 

Larger lift means more interesting rules. Association rules with high support are potentially interesting rules. Similarly, rules with high confidence would be interesting rules as well. So, you can take the conclusion with the highest the three parameters in that association rules. 

On the contrary, since BREAD and TEA have 0.87 Lift Score (less than 1), meaning negative correlation, unlikely to be bough together. 


*	Lift (antecedent => consequent) = 1 means that there is no correlation within the itemset.

*	Lift (antecedent=> consequent) > 1 means that there is a positive correlation within the itemset, i.e., products in the itemset, antecedent, 
and consequent, are more likely to be bought together.

* Lift (antecedent=> consequent) < 1 means that there is a negative correlation within the itemset, i.e., products in itemset, antecedent, and consequent, are unlikely to be bought together.


# Finally, we will cross verify our results with the Standard Package available in Python named mlextend.frequent_patterns having Apriori and association rules modules.


In [31]:

#loading standard package
from mlxtend.frequent_patterns import apriori, association_rules

#findinf frequent itenset with min support = 4%

frequent_itemset = apriori(df = df , min_support = .2, use_colnames= True)
frequent_itemset.sort_values(by = 'support', ascending = False)

,support,itemsets
2,0.65,(BREAD)
3,0.40,(COFFEE)
0,0.35,(BISCUIT)
8,0.35,(TEA)
4,0.30,(CORNFLAKES)
7,0.30,(SUGER)
5,0.25,(MAGGI)
6,0.25,(MILK)
1,0.20,(BOURNVITA)
9,0.20,"(BISCUIT, BREAD)"


In [32]:
#Rule with minimun confidence 50%

Rules = association_rules(frequent_itemset, min_threshold = 0.5)
Rules.sort_values(by = 'confidence', ascending = False)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(MILK),(BREAD),0.25,0.65,0.2,0.800000,1.230769,0.0375,1.750000
9,(MAGGI),(TEA),0.25,0.35,0.2,0.800000,2.285714,0.1125,3.250000
2,(SUGER),(BREAD),0.30,0.65,0.2,0.666667,1.025641,0.0050,1.050000
4,(CORNFLAKES),(COFFEE),0.30,0.40,0.2,0.666667,1.666667,0.0800,1.800000
7,(SUGER),(COFFEE),0.30,0.40,0.2,0.666667,1.666667,0.0800,1.800000
0,(BISCUIT),(BREAD),0.35,0.65,0.2,0.571429,0.879121,-0.0275,0.816667
3,(TEA),(BREAD),0.35,0.65,0.2,0.571429,0.879121,-0.0275,0.816667
8,(TEA),(MAGGI),0.35,0.25,0.2,0.571429,2.285714,0.1125,1.750000
5,(COFFEE),(CORNFLAKES),0.40,0.30,0.2,0.500000,1.666667,0.0800,1.400000
6,(COFFEE),(SUGER),0.40,0.30,0.2,0.500000,1.666667,0.0800,1.400000


### From here we also get, 

###(MILK)	(BREAD)	Confidence=  0.800000, Lift = 	1.230769 

###and 

###(MAGGI)	(TEA)	Confidence = 0.800000, Lift = 2.285714

###Therefor, we can see that we derived the same result as the library function. 
